# Compile Woodstock-style input files

This notebook was created by Jimmy Ke and Greg Paradis. 

The model is implemented using `ws3`. We use `Woodstock`-style text input files to compile the basic input data for our `ws3` model (i.e., stratification schema, initial inventory, yield curves, treatments, transitions). 

## Forest Estate Model Input File Generation


To simplify the knowledge required for programming, we use formatted files to support the forest estate model. This script will generate the following files needed for the forest estate model:
- Landscape file
- Area file
- Growth and yield file
- Forest action file
- Transition rule file

In [4]:
!git submodule init
!git submodule update

The following is a hack to force this notebook to use the local `ws3` submodule code.

In [5]:
import os
import sys
module_path = os.path.abspath(os.path.join('ws3'))
if module_path not in sys.path:
    sys.path.append(module_path)
sys.path

['/media/data/project/gparadis/model_prototype_0',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/opt/jupyterhub/lib/python3.10/site-packages',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/media/data/project/gparadis/model_prototype_0/ws3']

## Preamble

Import modules, define classes and functions and constants, etc.

In [1]:
import pandas as pd
import geopandas as gpd
import ws3.forest, ws3.core

/opt/jupyterhub/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [4]:
data_path = 'data/input'
fert_codes = ['00',
              '11',
              '12',
              '13',
              '14',
              '15',
              '16',
              '17',
              '21',
              '22',
              '23',
              '24',
              '25',
              '26',
              '31',
              '32',
              '33',
              '34',
              '35']
period_length = 10
max_age = 350
base_year = 2020
horizon = 10
curve_table = pd.read_csv('%s/curve_table.csv' % data_path)
model_name = 'tsa24_clipped'

# Compile LANDSCAPE section

Hand coded to `data_path/model_name.lan`.

# Compile AREAS section

In [46]:
stands = gpd.read_file('%s/shp/tsa24_clipped.shp/stands.shp' % data_path)

Need to rebuild the THLB attribute in the stand inventory table. Current implementation is inconsistent with yield VDYP/TIPSY yield curve (and AU definition) modelling assumptions. 

In a nutshell, AUs either have only a VDYP yield curve or both VDYP and TISPY yield curves (if considered potentially operable). We need to set the THLB attribute to 0 for stands in AUs with have only a VDYP curve (i.e., where `au_table.unmanaged_curve_id == au_table.managed_curve_id`), and 1 otherwise. 

Otherwise, we would get weird cases where we can harvest a stand but there is not TIPSY curve defined for second-growth stand conditions.

In [47]:
au_table = pd.read_csv('%s/au_table.csv' % data_path).set_index('au_id')
curve_table = pd.read_csv('%s/curve_table.csv' % data_path)
curve_points_table = pd.read_csv('%s/curve_points_table.csv' % data_path).set_index('curve_id')

In [48]:
au_table = au_table.query('tsa == 24')

In [49]:
au_table['thlb'] = au_table.apply(lambda row: 0 if row.unmanaged_curve_id == row.managed_curve_id else 1, axis=1)

In [50]:
stands.theme1 = stands.apply(lambda row: au_table.loc[row.theme2].thlb, axis=1)

Copy `curve1` to `theme4` so we can track yield curve transitions independently from AU.

In [51]:
stands['theme4'] = stands.curve1

Set up themes.

In [52]:
theme_cols=['theme0', # TSA 
            'theme1', # THLB
            'theme2', # AU
            'theme3', # leading species code
            'theme4'] # yield curve ID

In [53]:
gstands = stands.groupby(theme_cols+['age'])

In [93]:
lines = []
for name, group in gstands:
    dtk, age, area = tuple(map(lambda x: str(x), name[:-1])), int(name[-1]), group['area'].sum()
    if dtk[1] == '0': continue
    if dtk[4][2] == '0':
        t4 = list(dtk[4])
        t4[2] = '2'
        t4 = ''.join(t4)
        dtk = (dtk[0], dtk[1], dtk[2], dtk[3], t4)
        print(dtk)
    #print(name, group)
    #print(dtk, age, area)
    #print('*A', ' '.join(v for v in dtk), age, area)
    lines.append('*A %s %i %0.1f\n' % (' '.join(v for v in dtk), age, area))
    
filename = '%s/%s.are' % (data_path, model_name)
print(filename)
with open(filename, 'w') as snk:
    snk.writelines(lines)

('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2402000', '100', '2422000')
('tsa24_clipped', '1', '2402002', '204', '2422002')
('tsa24_clipped', '1', '2402002', '204', '2422002')
('tsa24_clipped', '1', '2402002', '204', '2422002')
('tsa24_clipped', '1', '2402002', '204', '2422002')
('tsa24_clip

# Compile YIELDS section

In [94]:
lines = []
# for fert
for fert in fert_codes:
    au_table = pd.read_csv('%s/au_table_%s.csv' % (data_path, fert)).set_index('au_id')
    au_table = au_table.query('tsa == 24')
    au_table['thlb'] = au_table.apply(lambda row: 0 if row.unmanaged_curve_id == row.managed_curve_id else 1, axis=1)
    curve_points_table = pd.read_csv('%s/curve_points_table_updated_%s.csv' % (data_path, fert)).set_index('curve_id')
    for au_id, au_row in au_table.iterrows():
        if au_row.thlb == 0:
            fert_curve_id = au_row.fert_curve_id #apply curve id 
            managed_curve_id = au_row.managed_curve_id
            curve_id = au_row.managed_curve_id 
            mask = ('?', '?', str(au_id), '?', str(managed_curve_id))
            continue # (hack) omit unmanaged, i.e., NTHLB, AUs from the model
        else:
            fert_curve_id = au_row.fert_curve_id #apply curve id 
            managed_curve_id = au_row.managed_curve_id
            curve_id = au_row.fert_curve_id
            mask = ('?', '?', str(au_id), '?', str(fert_curve_id))
        lines.append('*Y %s\n' % ' '.join(v for v in mask))
        for i, col in enumerate(curve_points_table.columns):
            if i == 0:
                continue 
            yname = col
            points = [(r.age, r[col]) for _, r in curve_points_table.loc[curve_id].iterrows() if not r.age % period_length and r.age <= max_age]
            is_volume = True if yname == 'volume' else False
            c = ws3.core.Curve(yname, points=points, type='a', is_volume=is_volume, xmax=max_age, period_length=period_length)
            if not au_row.thlb: continue
            lines.append('%s 1 %s\n' % (yname, ' '.join(str(int(c[x])) for x in range(0, 300, 1))))
filename = '%s/%s.yld' % (data_path, model_name)
print(filename)
with open(filename, 'w') as snk:
    snk.writelines(lines)

data/input/tsa24_clipped.yld


# Compile ACTIONS section

In [122]:
lines = []

lines.append('*ACTION cc Y clearcut harvest\n')
lines.append('*OPERABLE cc\n')
lines.append('? 1 ? ? ? _AGE >= 40 AND _AGE <= 999\n')

for i in range(1,8):
    lines.append('*ACTION f1%i N fertilize once at age %i0\n' % (i,i))
    lines.append('*OPERABLE f1%i\n' % i)
    lines.append('? 1 ? ? unfertilized _AGE >= %i0 AND _AGE <= %i0\n' % (i,i))
for i in range(1,7):
    lines.append('*ACTION f2%i N fertilize twice at age %i0 %i0\n' % (i,i,i+1))
    lines.append('*OPERABLE f2%i\n' % i)
    lines.append('? 1 ? ? unfertilized _AGE >= %i0 AND _AGE <= %i0\n' % (i,i))
for i in range(1,6):
    lines.append('*ACTION f3%i N fertilize thrice at age %i0 %i0 %i0\n' % (i,i,i+1,i+2))
    lines.append('*OPERABLE f3%i\n' % i)
    lines.append('? 1 ? ? unfertilized _AGE >= %i0 AND _AGE <= %i0\n' % (i,i))
    
filename = '%s/%s.act' % (data_path, model_name)
print(filename)
with open(filename, 'w') as snk:
    snk.writelines(lines)

data/input/tsa24_clipped.act


# Compile TRANSITIONS section

In [96]:
lines = []

acode = 'cc'
lines.append('*CASE %s\n' % acode)
for au_id, au_row in au_table.iterrows():
    if not au_row.thlb: continue
    curve_id = au_row.managed_curve_id
    target_curve_id = au_row.managed_curve_id
    smask = ' '.join(('?', '?', str(au_id), '?', '?'))
    tmask = ' '.join(('?', '?', '?', '?', str(target_curve_id)))
    lines.append('*SOURCE %s\n' % smask)
    lines.append('*TARGET %s 100\n' % tmask)
    
fert_once = list(range(11, 18))
fert_twice = list(range(21,27))
fert_thrice = list(range(31,36))
fert_option_all = fert_once + fert_twice + fert_thrice
for i in fert_option_all:
    acode = 'f%i' % i
    lines.append('*CASE %s\n' % acode)
    for au_id, au_row in au_table.iterrows():
        if not au_row.thlb: continue
        curve_id = au_row.managed_curve_id
        target_curve_id = curve_id + i * 10
        smask = ' '.join(('?', '?', str(au_id), '?', '?'))
        tmask = ' '.join(('?', '?', '?', '?', str(target_curve_id)))
        lines.append('*SOURCE %s\n' % smask)
        lines.append('*TARGET %s 100\n' % tmask)
    
filename = '%s/%s.trn' % (data_path, model_name)
print(filename)
with open(filename, 'w') as snk:
    snk.writelines(lines)

data/input/tsa24_clipped.trn
